### imports and preparation

- place this notebook in ../typeform 
- store your own token for tf API in a file called "typeform_token.py" and save it locally (do not commit) in /typeform
- The file contains only one line: TOKEN="your_token"

In [ ]:
ls

In [ ]:
import requests
import pandas as pd
#import my own function
%run one_entry_to_dictionary.py
%run typeform_token.py

# 1. Pull Data from Typeform

### Connect to TF, see metainfo for form

In [ ]:
#IMPORT TOKEN FILE 
auth_token= TOKEN
head = {'Authorization': 'Bearer ' + auth_token}
url = 'https://api.typeform.com/forms/OHziDn'

metainfo_form = requests.get(url, headers=head)
#response 200 means its good !
print(metainfo_form)
print(metainfo_form.json())

### Pull all responses exclusing mail from Typeform

In [ ]:
# get all answers but not emails
SIZE = "500"
url = "https://api.typeform.com/forms/OHziDn/responses?fields=Xa2T0bcZ5VnY,QUtA6k0gcrZf,ZNZyNVwMtwID,azlKRkvdcc7O,Vrzj2uv0i29C,Xh2W1d6FhXyK,t9EaTsONnJfR,OaujoVSY6NaS,ASolp4v3adZK,erHBE6gKxycf,ASQmq4h3vxRt,KEGztC8iYfyy,kWb7LUjdJyhb,g9goY6hIxmC9,jG2wzvAmOWxq,HNqA7BK7ZDFl&page_size="+SIZE

response = requests.get(url, headers=head)
#response 200 means its good !
print(response)

responses = response.json()

In [ ]:
print("number of responses",responses['total_items'])
#look at the first response
print("the last response:",responses["items"][0])

# 2. Transform JSON to pandas.df and map dummy vars

### step by step

In [ ]:
# LOAD ONE PERSON FROM THE JSON RESPONSE
one_person = responses["items"][0]

In [ ]:
#transform ONE person's information to a dictionary using function one_entry_to_dictionary
dictionary_test = one_entry_to_dictionary(one_person)
print(dictionary_test)

In [ ]:
# transform dictionary to df 
df_test = pd.DataFrame(dictionary_test,index=[0])
df_test.head()

### apply this to entire JSON

In [ ]:
def transform_all_to_df(responses):
    '''
    transforms the JSON response from the above query into a df with cols=vars, rows=individuals
    '''
    no_df_defined = True
    for item in responses["items"]:
        dictionary_transformation = one_entry_to_dictionary(item)
        df = pd.DataFrame(dictionary_transformation,index=[0])
        #crate dictionary and then append to it 
        if no_df_defined:
            df_all = df
            no_df_defined = False
        else: 
            df_all = df_all.append(df)
    return df_all

In [ ]:
df_all = transform_all_to_df(responses)

In [ ]:
df_all.head()

In [ ]:
len(df_all)


# Aggregation for Graph 1. Scatter-Testimonial


In [ ]:
df_graph_1 = df_all[["submitted_at","testimonial"]]

In [ ]:
df_graph_1.to_csv("data_graph1.csv")